In [9]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import csv
import pandas as pd
import pyttsx3
import joblib
import time

# Definitions

In [10]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [11]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [12]:
def draw_styled_landmarks_G(image, results):
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness = 2,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness = 2,circle_radius=1)
                             )


def draw_styled_landmarks_np_nf_B(image, results):
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,0,0), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness = 2,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,0,0), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness = 2,circle_radius=1)
                             )

## Voice

In [13]:
def speak(text):
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')
    engine.setProperty('rate', 150)

    #Setting the voice
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)

    #Text input
    engine.say(text)
    engine.runAndWait()

# Make Detections

In [14]:
model_L = joblib.load('MP_model_head.pkl')

In [15]:
def sign_output(sign_list, sentence, sentence_out):
    with open('multi_sign.csv') as multisign_file:
        sign_list = csv.reader(multisign_file)
        for row in sign_list:
            if sentence[-1] == row[-1]:
                if sentence[-2] == row[-2]:
                    sentence_out.append(row[0])
                    break
            else:
                continue

In [16]:
def detect(vidsource):
    
    sentence = []
    sentence_out = []
    
    predictions = []
    
    last_sign_list = []
    one_sign_list = []
    
    #minimum probability
    threshold = 0.92
    
    #minimum number of predictions for confirmation
    pr = 3
    
    #for fps calculation
    pTime = 0
    cTime = 0
    
    #Loading complex signs mechanism
    with open('multi_sign.csv') as multisign_file:
        sign_list = csv.reader(multisign_file)
        for row in sign_list:
            last_sign_list.append(row[-1])
    
    #Loading simple signs
    #with open('single_sign.csv') as singlesign_file:
    #    singlesign_list = csv.reader(singlesign_file)
    #    for row in singlesign_list:
    #        one_sign_list.append(row[0])
    
    #Detecting from source of video feed
    cap = cv2.VideoCapture(vidsource)
    
    #set mediapipe model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        while cap.isOpened():
        #for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):

            #read frame
            ret, frame = cap.read()

            #make detections
            image, results = mediapipe_detection(frame, holistic)
            
            #Draw for tracking
            draw_styled_landmarks_np_nf_B(image, results)

            #Extract landmarks
            head =list(np.zeros(1*3))
            lh_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3))
            rh_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3))
            
            if results.pose_landmarks:
                for id, lm in enumerate(results.pose_landmarks.landmark):
                    h,w,c = frame.shape
                    cx, cy = int(lm.x*w), int(lm.y*h)
         
                    if id == 0:
                        if lm.visibility > 0.8:
                            head = list(np.array([lm.x, lm.y, lm.z]))
                        else:
                            head =list(np.zeros(1*3))
                      
            
            #Concatenate rows
            row = lh_row + rh_row + head

            #Make Detections
            X = pd.DataFrame([row])
            sign_class = model_L.predict(X)[0]
            sign_prob = model_L.predict_proba(X)[0]

            #Sentence Logic
            if sign_prob[np.argmax(sign_prob)] > threshold:
                predictions.append(sign_class)

                print(sign_class, sign_prob[np.argmax(sign_prob)])


                if predictions[-pr:] == [sign_class]*pr:
                    if len(sentence) > 0:
                        if sign_class != sentence[-1]:
                            sentence.append(sign_class)
                            
                            #Shows that a detection has been made
                            draw_styled_landmarks_G(image, results)
                            
                            #checks if a sign is a complex sign(last sign of the complex sign)
                            if sentence[-1] in last_sign_list:
                                sign_output(sign_list, sentence, sentence_out)
                            
                            #Checks if a sign is a simple sign
                            if sentence[-1] in one_sign_list:
                                sentence_out.append(sign_class)

                            speak(sign_class)
                    else:
                        sentence.append(sign_class)
                        draw_styled_landmarks_np_nf_B(image, results)
                        if sentence[-1] in one_sign_list:
                                sentence_out.append(sign_class)
                        speak(sign_class)


            if len(sentence) > 5:
                    sentence = sentence[-5:]
                    
                    
            if len(sentence_out) > 6:
                    sentence_out = sentence_out[-6:]
                    #speak(sentence_out)

            cv2.rectangle(image, (0,0), (640,40),(0,0,0), -1 )
            cv2.putText(image,  ' '.join(sentence), (3,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.rectangle(image, (0,80), (640,40),(255,0,0), -1 )
            cv2.putText(image,  ' '.join(sentence_out), (3,70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

            #fps
            cTime = time.time()
            fps = 1/ (cTime - pTime)
            pTime = cTime

            cv2.putText(image,"fps",(5,415), cv2.FONT_HERSHEY_PLAIN,3,(0,0,0),2)
            cv2.putText(image,str(int(fps)),(10,460), cv2.FONT_HERSHEY_PLAIN,3,(0,0,0),5)

   
            #show to screen
            cv2.imshow('OpenCV Feed', image)

            #break loop
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
detect(0)

  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9999251195456625
  0.999976921104665
  0.999965106153545
  0.9999742621417916
  0.9999801170133745
  0.9999727889033598
  0.999975323365359
  0.9999675967742992
  0.9999355994375014
  0.99991962304823
  0.9999180681612606
  0.9999504053374678
  0.9999582061152303
  0.9999562180507036
  0.9999440066856211
  0.9999524303997367
  0.999906056426917
  0.9999609662861986
  0.9999893023555696
  0.9999896718955255
  0.9999917315769672
  0.9999762719672074
  0.9999774875768739
  0.9999623247960269
  0.999962445815133
  0.9999435670337452
  0.9999312062376472
  0.9999668091616933
  0.999921656311568
  0.99995260116835
  0.9999187248032579
  0.9999784463007495
  0.9999823550678568
  0.9999174873717618
  0.9999102635703859
  0.999575420692964
  0.9998542351092963
  0.9998489845307849
  0.999549120005969
  0.9990557228996101
  0.9996211094413723
  0.9996219697018581
  0.9998692723589007
  0.9998628384235506
  0.

  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.992535318

  0.9999727480100715
  0.9999648873555622
  0.9999799000225988
  0.9999769683549903
  0.9999744552991556
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9999915901006959
  0.9925353185459393
  0.999990942590769
  0.9925353185459393
  0.9999825396990016
  0.9925353185459393
  0.9999925846058666
  0.999985841850302
  0.9925353185459393
  0.9925353185459393
  0.9999746146513866
  0.9999641298584903
  0.9925353185459393
  0.9999419652303465
  0.9999793303552954
  0.9999766935195474
  0.9999856360903677
  0.9999819904674926
  0.9999894428798419
  0.9999854990647715
  0.9999868002325825
  0.9999858783476211
  0.9999878687399012
  0.9999803391272911
  0.9999897395045187
  0.9999898608936247
  0.9999921415101172
  0.9999904613437239
  0.9999723436105379
  0.9999638879903019
  0.9999716617673943
  0.9999719967947117
  0.9999767783641897
  0.9999651310253059
  0.9999875487001105
  0.9925353185459393
  0.9925353185459393
  0.99999113697

  0.9976282126385366
  0.9968449000253519
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9765929196153363
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.992535318

  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9999933593662389
  0.9999883908767182
  0.9999802435918173
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.992535318

...

  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.992535318

  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.992535318

  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.992535318

  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.992535318

  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.9925353185459393
  0.992535318